In [45]:
import os
import re
import json
import copy
import time
import random
import pickle
import spacy
import numpy as np

from utils.chat2DeepSeek import Chat2DeepSeek
from utils.gen_prompt import Prompt
from utils.util import *

nlp = spacy.load("en_core_web_sm")

In [46]:
from collections import Counter

def unique_sorted_by_frequency(lst):
    counter = Counter(lst)
    # 按频率降序，然后按元素值升序
    return sorted(counter.keys(), key=lambda x: (-counter[x], x))

# 示例
my_list = [2, 3, 4, 2, 1, 3, 2, 4, 4, 4]
result = unique_sorted_by_frequency(my_list)
print(result)  # 输出: [4, 2, 3, 1]

[4, 2, 3, 1]


In [124]:
API_KEY = 'sk-553325d9958b4b3ba7bf15d297abb364'
event_dict = load_json('./meta_data/event_dict_full.json')
tpool = {}

In [126]:
# scout process
from pandas import Series

In [128]:
tpool = load_json('./exp_data/scout_trigger_7_2_4_42.json')

In [130]:
practical_roles = load_json('./meta_data/argument_pool_ori.json')
for event in practical_roles:
    for role in practical_roles[event]:
        practical_roles[event][role] = []
save_pickle(practical_roles, './exp_data/practical_roles.pkl')

In [132]:
practical_roles = load_pickle('./exp_data/practical_roles.pkl')

In [134]:
trigger_pool = {}
arguments_114 = []
for k, v in tpool.items():
    for e in v:
        event = e.replace('{', '').replace('}', '')
        if event == 'arguments_114':
            arguments_114.extend(v[event]) 
            continue
        if not trigger_pool.get(event):
            trigger_pool[event] = []
        trigger_pool[event].extend(v[event])

In [136]:
# 去重
for e, v in trigger_pool.items():
    l = []
    # s = Series(trigger_pool[e]).drop_duplicates().dropna().to_list()
    pool = Series(trigger_pool[e]).dropna().to_list()
    sorted_pool = unique_sorted_by_frequency(pool)
    if len(sorted_pool) > 10:
        s = sorted_pool[:10]
    else:
        s = sorted_pool
    for t in s:
        if len(t.split(' ')) > 1:
            continue
        l.append(t)
    trigger_pool[e] = l


In [138]:
s = {}
arg_pool = {}
for r_a in arguments_114:
    for event, args in r_a.items():
        if not arg_pool.get(event):
            arg_pool[event] = {}
        try:
            for role, arg_l in args.items():
                if role not in practical_roles[event]:
                    if not s.get(event):
                        s[event] = []
                    if role not in s[event]:
                        s[event].append(role)
                    continue
                if not arg_pool[event].get(role):
                    arg_pool[event][role] = []
                if arg_l:
                    arg_pool[event][role].extend(arg_l)
        except:
            print(role, args)

Place [{'Buyer': ['Ellison'], 'Seller': ['employee'], 'Artifact': ['company'], 'Place': []}, {'Buyer': ['Cola'], 'Seller': [], 'Artifact': ['franchise'], 'Place': []}]
Place None


{'TRANSFER-MONEY': ['Amount', 'Money'],
 'SUE': ['Other'],
 'SENTENCE': ['Sentence', 'Crime', 'Victims'],
 'FINE': ['Money'],
 'CHARGE-INDICT': ['Crime', 'Organization', 'Victim', 'Time'],
 'DIVORCE': ['Place'],
 'ELECT': ['Position', 'Year'],
 'START-POSITION': ['Position'],
 'EXTRADITE': ['Person'],
 'CONVICT': ['Crime', 'Victims', 'Sentence', 'Victim', 'Prosecutor'],
 'TRIAL-HEARING': ['Outcome', 'Charge'],
 'EXECUTE': ['Crime', 'Sentence'],
 'APPEAL': ['Defendant', 'Charge'],
 'RELEASE-PAROLE': ['Agent']}

In [142]:
# 去重

for event in arg_pool:
    for role in arg_pool[event]:
        pool = Series(arg_pool[event][role]).dropna().to_list()
        try:
            sorted_pool = unique_sorted_by_frequency(pool)
            if len(sorted_pool) > 40:
                s = sorted_pool[:len(sorted_pool) // 4]
            elif len(sorted_pool) > 20:
                s = sorted_pool[:len(sorted_pool) // 2]
            else:
                s = sorted_pool[:len(sorted_pool)]
            arg_pool[event][role] = s
        except:
            print(role, arg_pool[event][role])
        

In [144]:
arg_ori = load_json('./meta_data/argument_pool_ori.json')
for event in arg_ori:
    print(event)
    for role in arg_ori[event]:
        try:
            print('\t', role, len(arg_pool[event][role]), len(arg_ori[event][role]))
        except:
            print('\t missing: ', role, len(arg_ori[event][role]))

TRANSPORT
	 Vehicle 20 40
	 Artifact 50 151
	 Destination 45 204
	 Agent 44 44
	 Origin 20 90
	 missing:  Place 1
ELECT
	 Person 19 29
	 Entity 26 14
	 Place 11 17
START-POSITION
	 Person 35 43
	 Entity 39 28
	 Place 19 8
NOMINATE
	 Person 12 7
	 Agent 14 4
ATTACK
	 Place 81 160
	 Target 132 182
	 Attacker 91 183
	 Instrument 57 70
	 missing:  Victim 1
END-POSITION
	 Entity 18 51
	 Person 22 83
	 Place 18 10
MEET
	 Entity 78 136
	 Place 24 36
MARRY
	 Person 11 23
	 Place 12 4
PHONE-WRITE
	 Entity 30 66
	 Place 10 12
TRANSFER-MONEY
	 Giver 18 58
	 Recipient 19 55
	 Beneficiary 2 10
	 Place 15 6
SUE
	 Plaintiff 20 17
	 Defendant 13 20
	 Adjudicator 9 4
	 Place 8 2
DEMONSTRATE
	 Entity 23 20
	 Place 19 27
END-ORG
	 missing:  Place 3
	 Org 20 16
INJURE
	 Victim 27 35
	 Agent 14 17
	 Place 13 17
	 Instrument 14 16
DIE
	 Victim 58 141
	 Agent 29 72
	 Place 33 87
	 Instrument 18 20
	 missing:  Person 1
ARREST-JAIL
	 Person 29 36
	 Agent 17 13
	 Place 17 16
TRANSFER-OWNERSHIP
	 Buyer 21 42
	 A

In [146]:
for event in arg_pool:
    print(event)
    for role in arg_pool[event]:
        print('\t', role, len(arg_pool[event][role]))


START-POSITION
	 Person 35
	 Entity 39
	 Place 19
TRANSFER-OWNERSHIP
	 Buyer 21
	 Seller 17
	 Artifact 26
	 Place 20
MERGE-ORG
	 Org 15
DIE
	 Agent 29
	 Victim 58
	 Instrument 18
	 Place 33
TRANSFER-MONEY
	 Giver 18
	 Recipient 19
	 Beneficiary 2
	 Place 15
DECLARE-BANKRUPTCY
	 Org 14
NOMINATE
	 Person 12
	 Agent 14
ELECT
	 Person 19
	 Entity 26
	 Place 11
MEET
	 Entity 78
	 Place 24
ARREST-JAIL
	 Person 29
	 Agent 17
	 Place 17
ATTACK
	 Attacker 91
	 Target 132
	 Instrument 57
	 Place 81
END-POSITION
	 Person 22
	 Entity 18
	 Place 18
TRANSPORT
	 Agent 44
	 Artifact 50
	 Vehicle 20
	 Origin 20
	 Destination 45
SUE
	 Plaintiff 20
	 Defendant 13
	 Adjudicator 9
	 Place 8
PARDON
	 Defendant 2
	 Adjudicator 3
INJURE
	 Agent 14
	 Victim 27
	 Instrument 14
	 Place 13
DEMONSTRATE
	 Entity 23
	 Place 19
START-ORG
	 Agent 16
	 Org 18
	 Place 18
MARRY
	 Person 11
	 Place 12
PHONE-WRITE
	 Entity 30
	 Place 10
CONVICT
	 Defendant 11
	 Adjudicator 11
	 Place 5
END-ORG
	 Org 20
BE-BORN
	 Person 10


In [148]:
{i: len(v) for i, v in trigger_pool.items()}

{'START-POSITION': 10,
 'TRANSFER-OWNERSHIP': 10,
 'MERGE-ORG': 10,
 'DIE': 10,
 'TRANSFER-MONEY': 10,
 'DECLARE-BANKRUPTCY': 10,
 'NOMINATE': 10,
 'ELECT': 10,
 'MEET': 10,
 'ARREST-JAIL': 10,
 'ATTACK': 10,
 'END-POSITION': 10,
 'TRANSPORT': 10,
 'SUE': 10,
 'PARDON': 3,
 'INJURE': 10,
 'DEMONSTRATE': 10,
 'START-ORG': 9,
 'MARRY': 10,
 'PHONE-WRITE': 10,
 'CONVICT': 10,
 'END-ORG': 8,
 'BE-BORN': 10,
 'TRIAL-HEARING': 10,
 'EXECUTE': 8,
 'CHARGE-INDICT': 10,
 'SENTENCE': 10,
 'FINE': 10,
 'APPEAL': 10,
 'RELEASE-PAROLE': 10,
 'DIVORCE': 10,
 'EXTRADITE': 6,
 'ACQUIT': 5}

In [150]:
len(trigger_pool)

33

In [152]:
tpool_ori = load_json('./meta_data/trigger_pool_ori.json')
for i in tpool_ori:
    print(i, len(trigger_pool[i]), len(tpool_ori[i]))

TRANSPORT 10 192
ELECT 10 28
START-POSITION 10 43
NOMINATE 10 8
ATTACK 10 211
END-POSITION 10 57
MEET 10 46
MARRY 10 13
PHONE-WRITE 10 42
TRANSFER-MONEY 10 60
SUE 10 14
DEMONSTRATE 10 27
END-ORG 8 22
INJURE 10 42
DIE 10 85
ARREST-JAIL 10 22
TRANSFER-OWNERSHIP 10 43
EXECUTE 8 6
TRIAL-HEARING 10 17
SENTENCE 10 17
BE-BORN 10 13
CHARGE-INDICT 10 14
START-ORG 9 16
CONVICT 10 11
DECLARE-BANKRUPTCY 10 7
RELEASE-PAROLE 10 7
FINE 10 9
PARDON 3 2
APPEAL 10 4
MERGE-ORG 10 6
EXTRADITE 6 2
DIVORCE 10 4
ACQUIT 5 3


In [156]:
save_json(trigger_pool, './exp_data/trigger_sout_top10.json')
save_json(arg_pool, './exp_data/argument_sout_top-one-fourth.json')